<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия." data-toc-modified-id="Логистическая-регрессия.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия.</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#Метод-опорных-векторов" data-toc-modified-id="Метод-опорных-векторов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Метод опорных векторов</a></span></li><li><span><a href="#Проверка-на-тестовой-выборке" data-toc-modified-id="Проверка-на-тестовой-выборке-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Проверка на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

В данном проекте стоит задача создания инструмента, который будет искать токсичные комментарии и отправлять их на модерацию для интернет-магазина "Викишоп".

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении есть набор данных с разметкой о токсичности правок.

Значение метрики качества *F1* построенно модели должно быть не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Загрузим необходимые библиотеки и модули.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# откроем файл и посмотрим на содержимое
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
# поправим индексы
df = df.reset_index(drop=True)
df.index

RangeIndex(start=0, stop=159292, step=1)

In [5]:
# посмотрим на распределение классов в целевой перемнной
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Есть дисбаланс классов, "токсичных" значений почти в 9 раз меньше. Это надо учесть при обучении моделей.

In [6]:
# создадим корпус
corpus = df['text'].values

# инициализируем лемматизатор
lemmatizer = WordNetLemmatizer()

# функция для определния part_of_speech-тагов, чтобы корректно лемматизировать каждое слово
def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# функция лемматизации и очистки текста от лишних символов
def preproc_text(text):
    text = re.sub(r"[^a-zA-Z' ]", " ", text)
    text = ' '.join(text.split()).lower()
    tagged = nltk.pos_tag(nltk.word_tokenize(text)) 
    tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), tagged)
    lemmatized =[]
    for word, tag in tagged:
        if tag is None:
            lemmatized.append(word)
        else:        
            lemmatized.append(lemmatizer.lemmatize(word, tag))
    return ' '.join(lemmatized)

tqdm.pandas()

df['lemm_text'] = df['text'].progress_apply(preproc_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [7]:
# проверим, что функция работает и каждое слово лемматизируется правильно.

sentence = "The striped bats are hanging on their feet for best"
preproc_text(sentence)

'the striped bat be hang on their foot for best'

In [8]:
# обработанные тексты - признаки, целевой признак - метка токситчный комментарий или нет
X = df['lemm_text']
y = df['toxic']

# выделяем обучающую и тестовую выборки, параметр stratify=y для балансировки классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123, stratify=y)
X_train.shape,  X_test.shape, y_train.shape, y_test.shape

((143362,), (15930,), (143362,), (15930,))

In [9]:
# инициализируем стоп-слова
stopwords = set(nltk_stopwords.words('english'))

# для преобразования текстов используем TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=list(stopwords), min_df=50)

# посмотрим на размер будущей матрицы, но трансформирование к выборкам применим в пайплайне вместе с моделью
vectorizer.fit_transform(X_train).shape

(143362, 5953)

## Обучение

Обучим несколько моделей, проверим их метрикой f1 на кросс-валидации, затем протестируем модель с наибольшим значением метрики на тестовой выборке.

### Логистическая регрессия.

In [10]:
%%time
# сoздадим пайплайн из векторайзера и модели
pipe_log_reg = Pipeline([('vectorizer', vectorizer),
                        ('clf', LogisticRegression(C=10, random_state=123))])

# метрика f1 на кросс-валидации
f1_log_reg = cross_val_score(pipe_log_reg, X_train, y_train, cv=5, scoring='f1').mean()

print('Значение f1 при обучении логистической регрессии на кросс-валидации:', round(f1_log_reg, 4))

Значение f1 при обучении логистической регрессии на кросс-валидации: 0.7658
CPU times: user 44.8 s, sys: 202 ms, total: 45 s
Wall time: 45.1 s


### LGBMClassifier

In [11]:
%%time
# пайплайн из векторайзера и модели
pipe_lgbm_clf = Pipeline([('vectorizer', vectorizer),
                        ('clf', LGBMClassifier(random_state=123))])

f1_lgbm = cross_val_score(pipe_lgbm_clf, X_train, y_train, cv=3, scoring='f1').mean()

print('Значение f1 LGBMClassifierf на кросс-валидации:', round(f1_lgbm, 3))

Значение f1 LGBMClassifierf на кросс-валидации: 0.74
CPU times: user 20min 27s, sys: 17.1 s, total: 20min 44s
Wall time: 20min 51s


### Метод опорных векторов

In [12]:
%%time
# используем гиперпараметры по умолчанию
pipe_svc = Pipeline([('vectorizer', vectorizer),
                ('clf', SVC(random_state=123))])

f1_svc = cross_val_score(pipe_svc, X_train, y_train, cv=3, scoring='f1').mean()

print('Значение f1 SVC на кросс-валидации:', round(f1_svc, 3))

Значение f1 SVC на кросс-валидации: 0.741
CPU times: user 1h 32min 58s, sys: 5.62 s, total: 1h 33min 3s
Wall time: 1h 33min 7s


In [13]:
# сравним метрики моделей на кросс-валидации
f1_all = pd.DataFrame({'модель': ['LogisticRegression', 'SVC', 'LGBMClassifier'],
                     'f1': [f1_log_reg, f1_lgbm, f1_svc]})
f1_all

,модель,f1
0,LogisticRegression,0.765795
1,SVC,0.740157
2,LGBMClassifier,0.740948


Как видим, логистическая регрессия дала наибольшее значение f1 на кросс-валидации при наименьшей скорости. Проверим ее на тестовой выборке. 

### Проверка на тестовой выборке

In [15]:
%%time
best_model = Pipeline([('vectorizer', vectorizer),
                        ('clf', LogisticRegression(C=10, random_state=123))])

best_model.fit(X_train, y_train)
pred = best_model.predict(X_test)

f1_final = f1_score(y_test, pred)

print('Значение f1 модели логистической регрессии на тестовой выборке:', round(f1_final, 3))

Значение f1 модели логистической регрессии на тестовой выборке: 0.772
CPU times: user 9.95 s, sys: 68 ms, total: 10 s
Wall time: 10 s


## Выводы

В целях определения тональности комментариев тексты были лемматизированы и очищены от лишних символов. Далее они были преобразованы в признаки с помощью TfidfVectorizer.

На этапе обучения моделей лучший результат метрики f1 показала модель логистической регрессии (f1 составило 0.7603) при наименьшей скорости. Модели SVC и LGBMClassifier показали примерно одинаковый результат метрики (примерно 0.73), также они значительно проигрывают по скорости. 

  Значение f1 модели на тестовой выборке составило 0.772.